In [1]:
from bs4 import BeautifulSoup
import requests
import pickle
import imdb_funcs
import json
import re

In [2]:
actor_file = open('popularActors.pkl', 'rb')

In [3]:
actor_list = pickle.load(actor_file)

In [4]:
# Change name of Robert Downey Jr. to Robert Downey
actor_list[1] = 'Robert Downey'

In [5]:
# Change name of Samuel L. Jackson to Samuel Jackson
actor_list[26] = 'Samuel Jackson'

In [6]:
#Use imdb_funcs.get_actor_titles to scrape for IMBD actor ID, URL, Name and IMDB movie ID and movie titles.
actor_dict_list = imdb_funcs.get_actor_titles(actor_list)

In [8]:
# Check total movies in the actor_dict_list - hence total number of web requests required to webscrape movies 
# tot = 2640
tot = 0
for i in actor_dict_list:
    tot += len(i['titles'])
    
print(tot)

2640


In [7]:
actor_dict_list

[{'actor_id': 'nm0262635',
  'name': 'Chris Evans',
  'url': 'https://www.imdb.com/name/nm0262635/',
  'titles': [('tt5697510', 'Jekyll'),
   ('tt8946378', 'Knives Out'),
   ('tt4995776', 'Operation Brothers'),
   ('tt4154796', 'Avengers: Endgame'),
   ('tt4154664', 'Captain Marvel'),
   ('tt4154756', 'Avengers: Infinity War'),
   ('tt2250912', 'Spider-Man: Homecoming'),
   ('tt4481414', 'Gifted'),
   ('tt3498820', 'Captain America: Civil War'),
   ('tt0478970', 'Ant-Man'),
   ('tt2395427', 'Avengers: Age of Ultron'),
   ('tt2139555', 'Playing It Cool'),
   ('tt0443465', 'Before We Go'),
   ('tt1843866', 'Captain America: The Winter Soldier'),
   ('tt1981115', 'Thor: The Dark World'),
   ('tt1706620', 'Snowpiercer'),
   ('tt1491044', 'The Iceman'),
   ('tt0848228', 'The Avengers'),
   ('tt0770703', "What's Your Number?"),
   ('tt1740721', 'Captain America: Super Soldier'),
   ('tt0458339', 'Captain America: The First Avenger'),
   ('tt1582248', 'Puncture'),
   ('tt0446029', 'Scott Pilg

In [11]:
# store data in a JSON file
# with open('imdb_actors.json', 'w') as jfile:
#     json.dump(actor_dict_list, jfile)

In [9]:
#'sub_list' dummy list created to imdb movie ID to be used at the end of the IMDB URL to webscrape the movies
sub_list = []
for item in actor_dict_list:
    for title in item['titles']:
        sub_list.append(title[0])

In [10]:
sub_list

['tt5697510',
 'tt8946378',
 'tt4995776',
 'tt4154796',
 'tt4154664',
 'tt4154756',
 'tt2250912',
 'tt4481414',
 'tt3498820',
 'tt0478970',
 'tt2395427',
 'tt2139555',
 'tt0443465',
 'tt1843866',
 'tt1981115',
 'tt1706620',
 'tt1491044',
 'tt0848228',
 'tt0770703',
 'tt1740721',
 'tt0458339',
 'tt1582248',
 'tt0446029',
 'tt0480255',
 'tt0465580',
 'tt0896031',
 'tt0421073',
 'tt0858486',
 'tt0489237',
 'tt0486576',
 'tt0448134',
 'tt0453556',
 'tt0449061',
 'tt0120667',
 'tt0476458',
 'tt0401420',
 'tt0337921',
 'tt0314498',
 'tt0364008',
 'tt0368915',
 'tt0362585',
 'tt0277371',
 'tt0221431',
 'tt4593746',
 'tt6534532',
 'tt2094116',
 'tt2461172',
 'tt6673612',
 'tt4154796',
 'tt4154756',
 'tt2250912',
 'tt3498820',
 'tt2395427',
 'tt1872194',
 'tt2883512',
 'tt1300854',
 'tt0848228',
 'tt1515091',
 'tt1231583',
 'tt1686782',
 'tt1228705',
 'tt0988045',
 'tt0821642',
 'tt0942385',
 'tt0800080',
 'tt1233205',
 'tt0371746',
 'tt0423977',
 'tt0338216',
 'tt0443706',
 'tt0422295',
 'tt04

In [14]:
# IMBD movie page webscraper
def imdb_movie_page_parser_v3(url):
    print(url)
    # request the url as defined in the url parameter above
    html_page = requests.get(url)
    # use beautiful soup to store content of website
    soup = BeautifulSoup(html_page.content, 'html.parser')
    #slice last 9 characters of URL string to get the movie ID
    movie_id = url[-9:]
    title = soup.findAll('div', {'class':'title_wrapper'})[0].find('h1').text.replace('\xa0', ' ').strip()
    director = soup.findAll('div', {'class':'credit_summary_item'})[0].text.replace('\n', '').strip().replace('Director:','')
    cast1 = soup.findAll('td')[2].text.replace('\n', '').strip()
    cast2 = soup.findAll('td')[6].text.replace('\n', '').strip()
    cast3 = soup.findAll('td')[10].text.replace('\n', '').strip()
    if len(soup.findAll('td')) > 14:
        cast4 = soup.findAll('td')[14].text.replace('\n', '').strip()
    else:
        cast4 = None
    # if there is not 5th or 6th cast member, the length of soup.findAll('td') list which contains all of the cast
    # is used to stop 'index out of range' errors
    if len(soup.findAll('td')) > 18:
        cast5 = soup.findAll('td')[18].text.replace('\n', '').strip()
    else:
        cast5 = None
    if len(soup.findAll('td')) > 22:
        cast6 = soup.findAll('td')[22].text.replace('\n', '').strip()
    else:
        cast6 = None
    
    if soup.findAll('div', {'class':'titleReviewBarItem'}):
        metascore = soup.findAll('div', {'class':'titleReviewBarItem'})[0].find('span').text
    else:
        metascore = None
    
    # the genres do not follow the same order within the findAll list initially parsed. As such the following is done
    genres_loc = soup.findAll('div', {'class':'see-more inline canwrap'}) #set the list container as a variable genres_loc
    x=0 #set x variable to zero to be used in the for loop to identify the location of the genres sub-container through its index number
    y=0 #set y variable to zero this will be used in the if statement below. Explaination of the statement to follow
    for i in range(len(genres_loc)): # i is the index number of the range-length of the genres_loc variable
        if str(genres_loc[i]).find('Genres:') > 0: # the keyword that always preceeds the genres sub-container is 'Genres:'
            x = i                                  #the .find() method returns a +ve number to indicate the location of the keyword
        else:                                      #if it does not find the keyword it will return -1 as output for the i-th container
            y-=1                                   #y variable add on the -1 values
    if y == len(genres_loc)*-1:                    #if the length of the main container, genres_loc, *-1 equals the y varialbe
        genres = None                              #the there is no specified genre and genres value is None
    else:                                          # else i will return the sub-container through the index number i and set it equal to the value of genres in the dictionary after trimming down the unwanted characters and spaces
        genres = soup.findAll('div', {'class':'see-more inline canwrap'})[x].text.strip().replace('\n','').replace('\xa0|',',').replace('Genres: ','').split(',')
        
    # for budget and box office numbers, the same problem is faced with the cast- thus same if statement is used
    if len(soup.findAll('div', {'class':'txt-block'})) > 9:
        budget = soup.findAll('div', {'class':'txt-block'})[9].next.next_sibling.next_sibling.replace('\n', '').replace('$', '').replace(',','').strip()
    else:
        budget = None
    if len(soup.findAll('div', {'class':'txt-block'})) > 11:
        bo_gu = soup.findAll('div', {'class':'txt-block'})[11].next.next_sibling.next_sibling.replace('\n', '').replace('$', '').replace(',','').strip()
    else:
        bo_gu = None
    if len(soup.findAll('div', {'class':'txt-block'})) > 10:
        bo_op = soup.findAll('div', {'class':'txt-block'})[10].next.next_sibling.next_sibling.replace('\n', '').replace('$', '').replace(',','').strip()
    else:
        bo_op = None
    if len(soup.findAll('div', {'class':'txt-block'})) > 12:
        bo_c = soup.findAll('div', {'class':'txt-block'})[12].next.next_sibling.next_sibling.replace('\n', '').replace('$', '').replace(',','').strip()
    else:
        bo_c = None
    
    studio_loc = soup.findAll('div', {'class':'txt-block'})
    k=0
    j=0
    for i in range(len(studio_loc)):
        if str(studio_loc[i]).find('Production Co:') > 0:
            j = i   
        else:
            k-=1
    if k == len(studio_loc)*-1:
        studio = None
    else:
        studio =  soup.findAll('div', {'class':'txt-block'})[j].next.next.next.next.next.next
    
    movie_dict={'movie_id':movie_id, 'title':title, 'director': director, 'cast': [cast1, cast2, cast3, cast4, cast5, cast6], 'genres':genres,
                   'metascore': metascore, 'budget': budget, 'box office gross usa': bo_gu, 
                'box office cumulative': bo_c, 'box office opening week': bo_op, 'studio': studio} #removed int() from all
    print(movie_dict)
    return movie_dict

In [ ]:
# function to create imdb URLs to scrape
# def imdb_movie_page_URL_creator(list_of_movie_urls):
#     movie_dic_list = []
#     for element in list_of_movie_urls:
#         movie_dic_list.append('https://www.imdb.com/title/'+element)
#     return movie_dic_list

In [ ]:
# list of all 2640 movie URLS
# url_list = imdb_movie_page_parser_v3_looper(sub_list)

In [ ]:
#loop imdb_movie_page_parser_v3 function above to get the 2640 movies (pageation) and append to final_list
# final_list = []
# for url in url_list:
#     final_list.append(imdb_movie_page_parser_v3(url))

In [ ]:
# Save final_list as a json file
# with open('imdb_movies_50_actors.json', 'w') as jfile:
#     json.dump(final_list, jfile)

In [2]:
#Import movies from the jason file
with open('json_files/imdb_movies_50_actors.json', 'r') as read_file:
    data = json.load(read_file)

In [3]:
data

[{'movie_id': 'tt5697510',
  'title': 'Jekyll',
  'director': 'Ruben Fleischer',
  'cast': ['Chris Evans',
   'Prime VideoUnlimited Streamingof Movies & TV',
   'Amazon FranceBuy Movies onDVD & Blu-ray',
   None,
   None,
   None],
  'genres': None,
  'metascore': '\n                    4,719\n      ( 150)\n                ',
  'budget': None,
  'box office gross usa': None,
  'box office cumulative': None,
  'box office opening week': None,
  'studio': ' A Very Good Production Inc.'},
 {'movie_id': 'tt8946378',
  'title': 'Knives Out (2019)',
  'director': 'Rian Johnson',
  'cast': ['Chris Evans',
   'Ana de Armas',
   'Daniel Craig',
   'Jamie Lee Curtis',
   'Michael Shannon',
   'Toni Collette'],
  'genres': ['Comedy', ' Crime', ' Drama', ' Mystery', ' Thriller'],
  'metascore': '\n                    293\n      ( 202)\n                ',
  'budget': '»',
  'box office gross usa': '',
  'box office cumulative': '1.85 : 1',
  'box office opening week': '',
  'studio': ' FilmNation E

In [ ]:
### DATA CLEANING

In [103]:
# 1. eliminating extra space in the list of genres sting (e.g. ' Action' -> 'Action')
for movie in data:
    if movie['genres'] != None:
        dummy_list = []
        for genre in movie['genres']:
            x = genre.strip()
            dummy_list.append(x)
            movie['genres'] = dummy_list
 

In [104]:
data1 = data
data1 #cleaned genres

[{'movie_id': 'tt5697510',
  'title': 'Jekyll',
  'director': 'Ruben Fleischer',
  'cast': ['Chris Evans',
   'Prime VideoUnlimited Streamingof Movies & TV',
   'Amazon FranceBuy Movies onDVD & Blu-ray',
   None,
   None,
   None],
  'genres': None,
  'metascore': '\n                    4,719\n      ( 150)\n                ',
  'budget': None,
  'box office gross usa': None,
  'box office cumulative': None,
  'box office opening week': None,
  'studio': ' A Very Good Production Inc.'},
 {'movie_id': 'tt8946378',
  'title': 'Knives Out (2019)',
  'director': 'Rian Johnson',
  'cast': ['Chris Evans',
   'Ana de Armas',
   'Daniel Craig',
   'Jamie Lee Curtis',
   'Michael Shannon',
   'Toni Collette'],
  'genres': ['Comedy', 'Crime', 'Drama', 'Mystery', 'Thriller'],
  'metascore': '\n                    293\n      ( 202)\n                ',
  'budget': '»',
  'box office gross usa': '',
  'box office cumulative': '1.85 : 1',
  'box office opening week': '',
  'studio': ' FilmNation Enter

In [105]:
# 2. Clean metascore to set unrated movies to 0 and the rated ones as integers
len(data1[1]['metascore'])

54

In [106]:
# making sure meta score is either a number or 0

for movie in data1:
    if movie['metascore'] == None: # If metascore is none, convert to zero
        movie['metascore'] = 0
    elif type(movie['metascore']) == str: # If its length is 2, convert to an integer
        if len(movie['metascore']) == 2:
            movie['metascore'] = int(movie['metascore'])
        elif len(movie['metascore']) > 2: # If greater than 2, which means it has picked up different information, set to 0
            movie['metascore'] = 0
                

In [108]:
data2 = data1

In [109]:
data2 #meta score and genres cleaned

[{'movie_id': 'tt5697510',
  'title': 'Jekyll',
  'director': 'Ruben Fleischer',
  'cast': ['Chris Evans',
   'Prime VideoUnlimited Streamingof Movies & TV',
   'Amazon FranceBuy Movies onDVD & Blu-ray',
   None,
   None,
   None],
  'genres': None,
  'metascore': 0,
  'budget': None,
  'box office gross usa': None,
  'box office cumulative': None,
  'box office opening week': None,
  'studio': ' A Very Good Production Inc.'},
 {'movie_id': 'tt8946378',
  'title': 'Knives Out (2019)',
  'director': 'Rian Johnson',
  'cast': ['Chris Evans',
   'Ana de Armas',
   'Daniel Craig',
   'Jamie Lee Curtis',
   'Michael Shannon',
   'Toni Collette'],
  'genres': ['Comedy', 'Crime', 'Drama', 'Mystery', 'Thriller'],
  'metascore': 0,
  'budget': '»',
  'box office gross usa': '',
  'box office cumulative': '1.85 : 1',
  'box office opening week': '',
  'studio': ' FilmNation Entertainment'},
 {'movie_id': 'tt4995776',
  'title': 'Operation Brothers (2019)',
  'director': 'Gideon Raff',
  'cast': 

In [ ]:
# 3. Clean the three box office and budget numbers to remove None or wrong data with 0 and convert numeric strings to integers

In [110]:
type(data2[3][keys[1]])

str

In [111]:
# process seperated to identify anamolies specific to each item
keys = ['budget', 'box office gross usa', 'box office cumulative', 'box office opening week']
for movie in data2:
    if movie[keys[0]] == None:
        movie[keys[0]] = 0
    elif type(movie[keys[0]]) == str and (movie[keys[0]]).isnumeric() == True:
        movie[keys[0]] = int(movie[keys[0]])
    elif type(movie[keys[0]]) == str and (movie[keys[0]]).isnumeric() == False:
        movie[keys[0]] = 0

In [112]:
for movie in data2:
    if movie[keys[1]] == None:
        movie[keys[1]] = 0
    elif type(movie[keys[1]]) == str and (movie[keys[1]]).isnumeric() == True:
        movie[keys[1]] = int(movie[keys[1]])
    elif type(movie[keys[1]]) == str and (movie[keys[1]]).isnumeric() == False:
        movie[keys[1]] = 0

In [113]:
for movie in data2:
    if movie[keys[2]] == None:
        movie[keys[2]] = 0
    elif type(movie[keys[2]]) == str and (movie[keys[2]]).isnumeric() == True:
        movie[keys[2]] = int(movie[keys[2]])
    elif type(movie[keys[2]]) == str and (movie[keys[2]]).isnumeric() == False:
        movie[keys[2]] = 0

In [114]:
for movie in data2:
    if movie[keys[3]] == None:
        movie[keys[3]] = 0
    elif type(movie[keys[3]]) == str and (movie[keys[3]]).isnumeric() == True:
        movie[keys[3]] = int(movie[keys[3]])
    elif type(movie[keys[3]]) == str and (movie[keys[3]]).isnumeric() == False:
        movie[keys[3]] = 0

In [116]:
# save progress in another jsonfile as back-up
# with open("data_file.json", "w") as write_file:
#     json.dump(data2, write_file)

In [4]:
with open('json_files/data_file.json', 'r') as read_file:
    data3 = json.load(read_file)

In [119]:
data4 = data3
data4 # Cleaned data- Integer box office and budget numbers, integer metascore and whitespace eliminated from genres

[{'movie_id': 'tt5697510',
  'title': 'Jekyll',
  'director': 'Ruben Fleischer',
  'cast': ['Chris Evans',
   'Prime VideoUnlimited Streamingof Movies & TV',
   'Amazon FranceBuy Movies onDVD & Blu-ray',
   None,
   None,
   None],
  'genres': None,
  'metascore': 0,
  'budget': 0,
  'box office gross usa': 0,
  'box office cumulative': 0,
  'box office opening week': 0,
  'studio': ' A Very Good Production Inc.'},
 {'movie_id': 'tt8946378',
  'title': 'Knives Out (2019)',
  'director': 'Rian Johnson',
  'cast': ['Chris Evans',
   'Ana de Armas',
   'Daniel Craig',
   'Jamie Lee Curtis',
   'Michael Shannon',
   'Toni Collette'],
  'genres': ['Comedy', 'Crime', 'Drama', 'Mystery', 'Thriller'],
  'metascore': 0,
  'budget': 0,
  'box office gross usa': 0,
  'box office cumulative': 0,
  'box office opening week': 0,
  'studio': ' FilmNation Entertainment'},
 {'movie_id': 'tt4995776',
  'title': 'Operation Brothers (2019)',
  'director': 'Gideon Raff',
  'cast': ['Chris Evans',
   'Haley

In [125]:
# change Robert Downey Jr. and Samuel L. Jackson wherever it appears in the cast list
for line in data4:
    for member in line['cast']:
        if member=='Robert Downey Jr.':
            x = line['cast'].index('Robert Downey Jr.')
            line['cast'][x] = 'Robert Downey'
        if member=='Samuel L. Jackson':
            x = line['cast'].index('Samuel L. Jackson')
            line['cast'][x] = 'Samuel Jackson'

In [126]:
#Final clean data with names changed
data4

[{'movie_id': 'tt5697510',
  'title': 'Jekyll',
  'director': 'Ruben Fleischer',
  'cast': ['Chris Evans',
   'Prime VideoUnlimited Streamingof Movies & TV',
   'Amazon FranceBuy Movies onDVD & Blu-ray',
   None,
   None,
   None],
  'genres': None,
  'metascore': 0,
  'budget': 0,
  'box office gross usa': 0,
  'box office cumulative': 0,
  'box office opening week': 0,
  'studio': ' A Very Good Production Inc.'},
 {'movie_id': 'tt8946378',
  'title': 'Knives Out (2019)',
  'director': 'Rian Johnson',
  'cast': ['Chris Evans',
   'Ana de Armas',
   'Daniel Craig',
   'Jamie Lee Curtis',
   'Michael Shannon',
   'Toni Collette'],
  'genres': ['Comedy', 'Crime', 'Drama', 'Mystery', 'Thriller'],
  'metascore': 0,
  'budget': 0,
  'box office gross usa': 0,
  'box office cumulative': 0,
  'box office opening week': 0,
  'studio': ' FilmNation Entertainment'},
 {'movie_id': 'tt4995776',
  'title': 'Operation Brothers (2019)',
  'director': 'Gideon Raff',
  'cast': ['Chris Evans',
   'Haley

In [127]:
# data4 saved into a json file
with open("cleaned_movie_data_for_50_actors.json", "w") as write_file:
    json.dump(data4, write_file)